In [1]:
import pandas as pd
import os

In [2]:
home = os.path.expanduser("~")
ar = pd.read_csv(home + "/data/CORE/rules_dump_20250806.csv")
ar.head(5)

,Core-ID,CDISC Rule ID,Error Message,Description,Standard Name,Standard Version,Scope,Executability,Status
0,NaN,DDF00034,The duration will vary (durationWillVary is no...,If duration will vary (attribute durationWillV...,USDM,4.0,"{""Entities"":{""Include"":[""Duration""]}}",Fully Executable,Draft
1,NaN,DDF00260,The id value contains a space.,Id values are expected not to have spaces in t...,USDM,4.0,"{""Entities"":{""Include"":[""ALL""]}}",Partially Executable - Possible Overreporting,Draft
2,CORE-000413,DDF00020,Population of otherReason is not consistent wi...,If the reason for a study amendment is 'Other'...,USDM,4.0,"{""Entities"":{""Include"":[""StudyAmendmentReason""]}}",Fully Executable,Draft
3,NaN,DDF00069,The combination of arm and epoch occurs more t...,Each combination of arm and epoch must occur n...,USDM,"3.0, 4.0","{""Entities"":{""Include"":[""StudyCell""]}}",Fully Executable,Draft
4,CORE-000795,FB1602,The Collection study day (--DY) is not populat...,When date/time of collection (--DTC) is popula...,"SDTMIG, SENDIG, SENDIG-GENETOX, SENDIG-DART, S...","3.2, 3.3, 3.4, 3.0, 3.1, 3.1.1, 1.0, 1.1, 1.2","{""Classes"":{""Include"":[""ALL""]},""Domains"":{""Inc...",Fully Executable,Published


In [3]:
ar['std'] = ar['Standard Name'].apply(lambda x: x.split(', ') if isinstance(x, str) else None)
ar['rids'] = ar['CDISC Rule ID'].apply(lambda x: x.split(', ') if isinstance(x, str) else None)
ar.head(5)

,Core-ID,CDISC Rule ID,Error Message,Description,Standard Name,Standard Version,Scope,Executability,Status,std,rids
0,NaN,DDF00034,The duration will vary (durationWillVary is no...,If duration will vary (attribute durationWillV...,USDM,4.0,"{""Entities"":{""Include"":[""Duration""]}}",Fully Executable,Draft,[USDM],[DDF00034]
1,NaN,DDF00260,The id value contains a space.,Id values are expected not to have spaces in t...,USDM,4.0,"{""Entities"":{""Include"":[""ALL""]}}",Partially Executable - Possible Overreporting,Draft,[USDM],[DDF00260]
2,CORE-000413,DDF00020,Population of otherReason is not consistent wi...,If the reason for a study amendment is 'Other'...,USDM,4.0,"{""Entities"":{""Include"":[""StudyAmendmentReason""]}}",Fully Executable,Draft,[USDM],[DDF00020]
3,NaN,DDF00069,The combination of arm and epoch occurs more t...,Each combination of arm and epoch must occur n...,USDM,"3.0, 4.0","{""Entities"":{""Include"":[""StudyCell""]}}",Fully Executable,Draft,[USDM],[DDF00069]
4,CORE-000795,FB1602,The Collection study day (--DY) is not populat...,When date/time of collection (--DTC) is popula...,"SDTMIG, SENDIG, SENDIG-GENETOX, SENDIG-DART, S...","3.2, 3.3, 3.4, 3.0, 3.1, 3.1.1, 1.0, 1.1, 1.2","{""Classes"":{""Include"":[""ALL""]},""Domains"":{""Inc...",Fully Executable,Published,"[SDTMIG, SENDIG, SENDIG-GENETOX, SENDIG-DART, ...",[FB1602]


In [4]:
unique_strings = list(pd.unique(ar['std'].explode().dropna()))
print(unique_strings)

# BEWARE - THIS IS A MANUALLY CREATED LIST FROM THE OUPUTS OF THE NEXT CELL
relevant_stds = {'SDTMIG', 'ADaMIG', 'Internal Standard', 'ADaMIG-MD', 'ADTTE', 'ADAMIG', 'Define-XML'}


['USDM', 'SDTMIG', 'SENDIG', 'SENDIG-GENETOX', 'SENDIG-DART', 'SENDIG-AR', 'TIG', 'ADaMIG', 'Internal Standard', 'ADaMIG-MD', 'ADTTE', 'OCCDS', 'ADAMIG', 'Model v2.1', 'Define-XML', 'SENDID-GENETOX']


In [5]:
counts = (
    ar["std"]
      .dropna()
      .map(lambda x: set(x))   # ensure per-row uniqueness
      .explode()
      .value_counts()
)

print(counts)

std
SDTMIG               1117
SENDIG                658
SENDIG-DART           649
SENDIG-AR             549
TIG                   360
SENDIG-GENETOX        299
USDM                  126
ADaMIG                 98
ADaMIG-MD              36
OCCDS                  14
ADAMIG                  4
SENDID-GENETOX          2
Internal Standard       1
ADTTE                   1
Model v2.1              1
Define-XML              1
Name: count, dtype: int64


In [6]:
mask = ar["std"].apply(
    lambda x: bool(relevant_stds.intersection(x)) if isinstance(x, (list, set, tuple)) else False
)

filtered_by_std = ar[mask]
filtered_by_std.head(10)

,Core-ID,CDISC Rule ID,Error Message,Description,Standard Name,Standard Version,Scope,Executability,Status,std,rids
4,CORE-000795,FB1602,The Collection study day (--DY) is not populat...,When date/time of collection (--DTC) is popula...,"SDTMIG, SENDIG, SENDIG-GENETOX, SENDIG-DART, S...","3.2, 3.3, 3.4, 3.0, 3.1, 3.1.1, 1.0, 1.1, 1.2","{""Classes"":{""Include"":[""ALL""]},""Domains"":{""Inc...",Fully Executable,Published,"[SDTMIG, SENDIG, SENDIG-GENETOX, SENDIG-DART, ...",[FB1602]
7,NaN,FB2305,A subject has a no record in EX but is assigne...,All subjects that have no record in the Exposu...,SDTMIG,3.2,"{""Classes"":{""Include"":[""SPECIAL PURPOSE""]},""Do...",Fully Executable,Draft,[SDTMIG],[FB2305]
20,CORE-000097,"CG0218, TIG0432",EPOCH values don't match between Subject Visit...,Raise an error when variable EPOCH values don'...,"SDTMIG, TIG","3.4, 3.2, 3.3, 1.0","{""Classes"":{""Include"":[""SPECIAL PURPOSE""]},""Do...",Partially Executable - Possible Overreporting,Published,"[SDTMIG, TIG]","[CG0218, TIG0432]"
21,CDISC.SDTMIG.CG0288,CG0288,TSVALCD is not a valid code in the version ide...,Raise an error when TSVCDREF ='CDISC' and TSVA...,SDTMIG,"3.4, 3.3, 3.2","{""Classes"":{""Include"":[""TRIAL DESIGN""]},""Domai...",Fully Executable,Draft,[SDTMIG],[CG0288]
22,CORE-000844,CG0531,RACE in DM equals 'MULTIPLE' but multiple race...,Part A - Raise an error when RACE in DM equals...,SDTMIG,"3.3, 3.4","{""Classes"":{""Include"":[""ALL""]},""Domains"":{""Inc...",Fully Executable,Published,[SDTMIG],[CG0531]
23,NaN,FB0611,Disposition date time value for DEATH record i...,Disposition date time value for DEATH record i...,SDTMIG,"3.2, 3.3, 3.4","{""Classes"":{""Include"":[""EVENTS""]},""Domains"":{""...",Fully Executable,Draft,[SDTMIG],[FB0611]
24,CORE-000259,"CG0171, FB0613",Date time of collection of subject status in S...,Date time of collection of subject status in S...,SDTMIG,"3.4, 3.3, 3.2","{""Classes"":{""Include"":[""FINDINGS""]},""Domains"":...",Fully Executable,Published,[SDTMIG],"[CG0171, FB0613]"
25,NaN,FB2301,A subject has a record in DM but not in EX whi...,All subjects in the Demographics domain who ar...,SDTMIG,3.2,"{""Classes"":{""Include"":[""SPECIAL PURPOSE""]},""Do...",Fully Executable,Draft,[SDTMIG],[FB2301]
26,NaN,FB2302,A subject has a record in DM but not in EX whi...,All subjects in the Demographics domain who ar...,SDTMIG,"3.3, 3.4","{""Classes"":{""Include"":[""SPECIAL PURPOSE""]},""Do...",Fully Executable,Draft,[SDTMIG],[FB2302]
28,CORE-000737,FB1102,The set of parameters without conditions liste...,The set of parameters without conditions liste...,SDTMIG,"3.2, 3.3, 3.4","{""Classes"":{""Include"":[""TRIAL DESIGN""]},""Domai...",Fully Executable,Draft,[SDTMIG],[FB1102]


In [7]:
# drop all old rules, they start with either "sd" or "ct"

mask = filtered_by_std["rids"].apply(
    lambda xs: not any(
        isinstance(s, str) and s.lower().startswith(("sd", "ct"))
        for s in xs
    ) if isinstance(xs, (list, tuple, set)) else True
)
filtered_by_rid = filtered_by_std[mask]
print(filtered_by_std.shape, filtered_by_rid.shape)

(1233, 11) (753, 11)


In [8]:
counts = (
    filtered_by_rid["std"]
      .dropna()
      .map(lambda x: set(x))   # ensure per-row uniqueness
      .explode()
      .value_counts()
)

print(counts)

std
SDTMIG               637
TIG                  280
ADaMIG                98
SENDIG                90
SENDIG-DART           89
SENDIG-GENETOX        87
SENDIG-AR             57
ADaMIG-MD             36
OCCDS                 14
ADAMIG                 4
Internal Standard      1
ADTTE                  1
Model v2.1             1
Define-XML             1
USDM                   1
Name: count, dtype: int64


In [9]:
filtered_by_rid.head(200)

,Core-ID,CDISC Rule ID,Error Message,Description,Standard Name,Standard Version,Scope,Executability,Status,std,rids
4,CORE-000795,FB1602,The Collection study day (--DY) is not populat...,When date/time of collection (--DTC) is popula...,"SDTMIG, SENDIG, SENDIG-GENETOX, SENDIG-DART, S...","3.2, 3.3, 3.4, 3.0, 3.1, 3.1.1, 1.0, 1.1, 1.2","{""Classes"":{""Include"":[""ALL""]},""Domains"":{""Inc...",Fully Executable,Published,"[SDTMIG, SENDIG, SENDIG-GENETOX, SENDIG-DART, ...",[FB1602]
7,NaN,FB2305,A subject has a no record in EX but is assigne...,All subjects that have no record in the Exposu...,SDTMIG,3.2,"{""Classes"":{""Include"":[""SPECIAL PURPOSE""]},""Do...",Fully Executable,Draft,[SDTMIG],[FB2305]
20,CORE-000097,"CG0218, TIG0432",EPOCH values don't match between Subject Visit...,Raise an error when variable EPOCH values don'...,"SDTMIG, TIG","3.4, 3.2, 3.3, 1.0","{""Classes"":{""Include"":[""SPECIAL PURPOSE""]},""Do...",Partially Executable - Possible Overreporting,Published,"[SDTMIG, TIG]","[CG0218, TIG0432]"
21,CDISC.SDTMIG.CG0288,CG0288,TSVALCD is not a valid code in the version ide...,Raise an error when TSVCDREF ='CDISC' and TSVA...,SDTMIG,"3.4, 3.3, 3.2","{""Classes"":{""Include"":[""TRIAL DESIGN""]},""Domai...",Fully Executable,Draft,[SDTMIG],[CG0288]
22,CORE-000844,CG0531,RACE in DM equals 'MULTIPLE' but multiple race...,Part A - Raise an error when RACE in DM equals...,SDTMIG,"3.3, 3.4","{""Classes"":{""Include"":[""ALL""]},""Domains"":{""Inc...",Fully Executable,Published,[SDTMIG],[CG0531]
...,...,...,...,...,...,...,...,...,...,...,...
356,CORE-000291,"CG0105, TIG0370",EXVAMT is present when EC exists.,"When EC exists, EXVAMT should not be used.","SDTMIG, TIG","3.4, 3.3, 3.2, 1.0","{""Classes"":{""Include"":[""INTERVENTIONS""]},""Doma...",Fully Executable,Published,"[SDTMIG, TIG]","[CG0105, TIG0370]"
360,CORE-000585,CG0313,One of following variables is not present in t...,Raise an error when one of the following varia...,SDTMIG,3.2,"{""Classes"":{""Include"":[""INTERVENTIONS"",""EVENTS...",Fully Executable,Published,[SDTMIG],[CG0313]
361,CORE-000200,"CG0348, TIG0513",'--ORRES cannot be null when --STAT is null or...,Verify that --ORRES is not missing when either...,"SDTMIG, TIG","3.4, 3.2, 3.3, 1.0","{""Classes"":{""Include"":[""FINDINGS""]},""Domains"":...",Fully Executable,Published,"[SDTMIG, TIG]","[CG0348, TIG0513]"
362,CORE-000571,"CG0024, TIG0307",LNKID variable is only found in one domain.,Verify that when --LNKID is present in one dom...,"SDTMIG, TIG","3.4, 3.3, 3.2, 1.0","{""Classes"":{""Include"":[""ALL""]},""Domains"":{""Inc...",Fully Executable,Published,"[SDTMIG, TIG]","[CG0024, TIG0307]"


In [10]:
# get SDTM rules
sdtm = filtered_by_rid[filtered_by_rid["std"].apply(lambda x: "SDTMIG" in x)].copy()
print('sdtm', sdtm.shape)

# get ADaM rules

wanted = {'ADaMIG', 'ADaMIG-MD', 'ADTTE', 'ADAMIG'}
mask = filtered_by_rid["std"].apply(
    lambda xs: (
        any(s in wanted for s in xs) if isinstance(xs, (list, tuple, set))
        else isinstance(xs, str) and xs in wanted
    )
)
adam = filtered_by_rid[mask].copy()
print('adam', adam.shape)

# get Define-XML rules
define = filtered_by_rid[filtered_by_rid["std"].apply(lambda x: "Define-XML" in x)].copy()
print('define', define.shape)

sdtm (637, 11)
adam (116, 11)
define (1, 11)


In [33]:
sdtm.to_csv(home + "/code/cdisc-rules-engine/tests/resources/rules/sdtm_rules.csv", index=False, header=True)
adam.to_csv(home + "/code/cdisc-rules-engine/tests/resources/rules/adam_rules.csv", index=False, header=True)
define.to_csv(home + "/code/cdisc-rules-engine/tests/resources/rules/define_rules.csv", index=False, header=True)